<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1344]:
import pandas as pd
import psycopg2
import requests # Дополнительно импортируем библиотеку requests
from bs4 import BeautifulSoup # Дополнительно импортируем библиотеку BeautifulSoup

In [1346]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [1347]:
# текст запроса
query_3_1 = f'''select
                    count(*) as cnt_vacancies
                from 
                    VACANCIES
'''

In [1348]:
# выодим результат
count_vacancies = pd.read_sql_query(query_3_1, connection)
count_vacancies

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/4124202279.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count_vacancies = pd.read_sql_query(query_3_1, connection)


,cnt_vacancies
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [1349]:
# текст запроса
query_3_2 = f'''select 
                    count(*) as cnt_employers
                from 
                    EMPLOYERS
'''

In [1350]:
# выводим результат
count_employers = pd.read_sql_query(query_3_2, connection)
count_employers

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/1210236520.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count_employers = pd.read_sql_query(query_3_2, connection)


,cnt_employers
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [1351]:
# текст запроса
query_3_3 = f'''select
                    count(*) as cnt_areas
                from
                    AREAS
'''

In [1352]:
# выводим результат
count_ares = pd.read_sql_query(query_3_3, connection)
count_ares

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/1969546299.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count_ares = pd.read_sql_query(query_3_3, connection)


,cnt_areas
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [1353]:
# текст запроса
query_3_4 = f'''select
                    count(*) as cnt_industries
                from
                    INDUSTRIES
'''

In [1354]:
# выводим результат
count_industries = pd.read_sql_query(query_3_4, connection)
count_industries

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/351031968.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count_industries = pd.read_sql_query(query_3_4, connection)


,cnt_industries
0,294


**Выводы по предварительному анализу данных:**
1. Количество вакансий - 49 197.
2. Количество работодателей - 23 501.
3. Количество регионов - 1362.
4. Количество сфер деятельности - 294.
5. Количество вакансий практически в двое больше количество работодателей, что может указывать на: (1) большинство работодателей размещают 1-2 вакансии, (2) на период потепления в той или иной сфере (выброс большого количества вакансий на агрегатор).
6. Достаточно широкий круг сфер деятельности, что говорит о хорошем присутствии работодателей на платформе и ее популярности.
7. Платформа пользуется спросов и в субъектах РФ.

***

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [1355]:
# текст запроса
query_3_5 = f'''select
                    a.name as areas,
                    count(*) as cnt
                from
                    VACANCIES as v
                    join AREAS as a on v.area_id = a.id
                group by a.name
                order by cnt desc
'''

In [1356]:
# выводим результат
cnt_vacancies_area = pd.read_sql_query(query_3_5, connection)
cnt_vacancies_area 

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/1571969483.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt_vacancies_area = pd.read_sql_query(query_3_5, connection)


,areas,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [1357]:
# текст запроса
query_3_6 = f'''select
                    count(*) as cnt
                from
                    VACANCIES
                where salary_from is not null or salary_to is not null 
'''

In [1358]:
# выводим результат
salary_count = pd.read_sql_query(query_3_6, connection)
salary_count

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/2617475886.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  salary_count = pd.read_sql_query(query_3_6, connection)


,cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [1359]:
# текст запроса
query_3_7 = f'''select
                    round(AVG(salary_from)) as mean_salary_from,
                    round(AVG(salary_to)) as mean_salary_to
                from
                    VACANCIES                
'''

In [1360]:
# выводим результат
mean_salaries = pd.read_sql_query(query_3_7, connection)
mean_salaries

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/3123732009.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mean_salaries = pd.read_sql_query(query_3_7, connection)


,mean_salary_from,mean_salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [1361]:
# текст запроса
query_3_8 = f'''select
                    schedule as schedule,
                    employment as employment,
                    count(*) as count
                from
                    VACANCIES
                group by
                    schedule,
                    employment
                order by count desc
'''

In [1362]:
# выводим результат
count_sch_emp = pd.read_sql_query(query_3_8, connection)
count_sch_emp

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/1850597055.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count_sch_emp = pd.read_sql_query(query_3_8, connection)


,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [1363]:
# текст запроса
query_3_9 = f'''select
                    experience as experience,
                    count(*) as count_vacancies
                from
                    VACANCIES
                group by experience
                order by count_vacancies asc
'''

In [1364]:
# выводим результат
experience_vacancies_count = pd.read_sql_query(query_3_9, connection)
experience_vacancies_count

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/2531584200.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  experience_vacancies_count = pd.read_sql_query(query_3_9, connection)


,experience,count_vacancies
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


**Выводы по детальному анализу вакансий:**
1. Москва, ожидаемо, занимает первое место по количеству вакансий (5333).
2. В топ-10, также ожидаемо, находятся города-миллионники.
3. Немного неожиданно было увидеть Минск, который по количеству вакансий занимает третье место. Вероятно, активная работа, проводимая в рамках привлечения специалистов Союзного государства.
4. От общего количества вакансий бОльший процент представлен в виде вакансий с заполненным хотя бы одним полем с зарплатой (24073).
5. Минимальная и максимальная вилка зарплат составляет от 71065 до 110537 соответственно.
6. Большинство соискателей указывают "полный день" в качестве желаемого рабочего графика с учетом полной занятости (35367). Меньше всего соискателей выбирают "полный день" для ведения проектной деятельности.
7. Полученные данные указывают нам на то, что большинство вакансий выкладываются для молодых специалистов с опытом работы от 1 года до 3 лет.

***

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [1365]:
# текст запроса
query_3_10 = f'''select
                     e.name as employers_name,
                     count(*) as cnt_vacancies
                 from
                     VACANCIES as v
                     join EMPLOYERS as e on e.id = v.employer_id
                 group by e.name
                 order by cnt_vacancies desc
                 limit 5  
'''

In [1366]:
# выводим результат
vacancies_employears = pd.read_sql_query(query_3_10, connection)
vacancies_employears

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/2834280485.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacancies_employears = pd.read_sql_query(query_3_10, connection)


,employers_name,cnt_vacancies
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [1367]:
# текст запроса
query_3_11 = f'''select
                     a.name as area,
                     count(e.id) as vacancies,
                     count(v.id) as employers
                 from
                     areas as a
                     left join employers as e on a.id = e.area
                     left join vacancies as v on a.id = v.area_id
                 where v.name is null
                 group by a.id
                 order by vacancies desc
'''

In [1368]:
# выводим результат
areas_count_vac_emp = pd.read_sql_query(query_3_11, connection)
areas_count_vac_emp

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/563669654.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  areas_count_vac_emp = pd.read_sql_query(query_3_11, connection)


,area,vacancies,employers
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
588,Скидель,0,0
589,Белоозерск,0,0
590,Орловский,0,0
591,Павловская,0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [1369]:
# текст запроса
query_3_12 = f'''select
                     e.name as employers_name,
                     count(distinct v.area_id) as areas_count
                 from
                     EMPLOYERS as e
                     left join VACANCIES as v on e.id = v.employer_id
                 group by e.name
                 order by count(distinct v.area_id) desc      
'''

In [1370]:
# выводим результат
employ_count_areas = pd.read_sql_query(query_3_12, connection)
employ_count_areas

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/1733493085.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employ_count_areas = pd.read_sql_query(query_3_12, connection)


,employers_name,areas_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [1371]:
# текст запроса
query_3_13 = f'''select
                     count(e.id) as count_employers
                 from
                     EMPLOYERS as e
                     left join EMPLOYERS_INDUSTRIES ei ON e.id = ei.employer_id
                     left join INDUSTRIES i ON ei.industry_id = i.id
                 where i.name is null 
'''

In [1372]:
# выводим результат
industry_empl = pd.read_sql_query(query_3_13, connection)
industry_empl

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/3038371480.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  industry_empl = pd.read_sql_query(query_3_13, connection)


,count_employers
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [1373]:
# текст запроса
query_3_14 = f'''select
                     e.name as company_name,
                     count(i.name) as count_industry
                 from
                     EMPLOYERS as e
                     left join EMPLOYERS_INDUSTRIES ei ON e.id = ei.employer_id
                     left join INDUSTRIES i ON ei.industry_id = i.id
                 group by e.name
                 having count(i.id) = 4
                 order by e.name asc  
'''

In [1374]:
# выводим результат
company_count_industry = pd.read_sql_query(query_3_14, connection)
company_count_industry

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/2682178414.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  company_count_industry = pd.read_sql_query(query_3_14, connection)


,company_name,count_industry
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4
...,...,...
1131,ЮРИОН,4
1132,ЮТИП Технологии,4
1133,ЯКласс,4
1134,ЯрНео,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [1375]:
# текст запроса
query_3_15 = f'''select
                     count(distinct e.id) as count_employers
                 from
                     EMPLOYERS as e
                     left join EMPLOYERS_INDUSTRIES as ei on e.id = ei.employer_id
                     left join INDUSTRIES as i on i.id = ei.industry_id
                 where i.name = 'Разработка программного обеспечения' 
'''

In [1376]:
# выводим результат
progr_support_count = pd.read_sql_query(query_3_15, connection)
progr_support_count

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/4191482095.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  progr_support_count = pd.read_sql_query(query_3_15, connection)


,count_employers
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

**Прошу объяснить, как это сделать, так как  понимаю не до конца (скорее совсем не пониманию)**

In [1377]:
# Дальше этого не понимаю
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России1'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
print(page.title)

<title>Города-миллионеры России1 — Википедия</title>


In [1378]:
# текст запроса

In [1379]:
# результат запроса

Выводы по анализу работодателей:
1. Яндекс является безоговорочным лидером по количеству вакансий (1933).
2. Можно проследить интересную международную закономерность - Республика Беларусь не входит даже в топ-5 регионов по количеству вакансий на российском сервисе HH. Второе место занимает Казахстан, что может говорить, как бы это ни звучало, о необходимости удешевление разного вида труда.
3. В отношении количества регионов, в которых работодатель публикует вакансии стоит отметить следующее - самые широкопубликуемые работодатели представляют телекоммуникационную, интернет или строительную сферы. Это, в свою очередь, обусловлено спецификой работы работодателя.
4. 8419 работодателей не указало сферу своей деятельности, что может говорить о некорректной модерации, старых записях, широком профиле деятельности.
5. Удивительно, что магазин спортивного питания 5lb имеет такое же количество сфер деятельности, как и разнооборазные IT или около IT компании.
6. Внушает оптимизм количество работодателей, у которых в сфере деятельности указано програмное обеспечение - 3553.   

***

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [1380]:
# текст запроса
query_3_16 = f'''select
                     count(v.name) as vacancies_name
                 from
                     VACANCIES as v
                 where (name ilike '%data%') or (name ilike '%данн%')
'''

In [1381]:
# выводим результат
data_vac_count = pd.read_sql_query(query_3_16, connection)
data_vac_count

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/2941714580.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_vac_count = pd.read_sql_query(query_3_16, connection)


,vacancies_name
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [1382]:
# текст запроса
query_3_17 = f'''select
                     count(v.name) as vacancies_jun_count
                 from
                     VACANCIES as v
                 where
                     (
                     (name ilike '%data scientist%') or
                     (name ilike '%data science%') or
                     (name ilike '%исследователь данных%') or
                     ((name ilike '%ML%') and (name not ilike '%HTML%')) or
                     (name ilike '%machine learning%') or
                     (name ilike '%машинн%обучен%') and 
                     (key_skills is not null)
                     )
                     and (v.name ilike '%junior%' or
                     v.experience ilike '%Нет опыта%' or
                     v.employment ilike '%Стажировка%')
                     
'''

In [1383]:
# выводим результат
jun_vac_count = pd.read_sql_query(query_3_17, connection)
jun_vac_count

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/4113016531.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  jun_vac_count = pd.read_sql_query(query_3_17, connection)


,vacancies_jun_count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [1384]:
# текст запроса
query_3_18 = f'''select
                     count(v.name) as vacancies_jun_count
                 from
                     VACANCIES as v
                 where
                     (
                     (name ilike '%data scientist%') or
                     (name ilike '%data science%') or
                     (name ilike '%исследователь данных%') or
                     ((name ilike '%ML%') and (name not ilike '%HTML%')) or
                     (name ilike '%machine learning%') or
                     (name ilike '%машинн%обучен%') and 
                     (key_skills is not null)
                     )
                     and (v.key_skills ilike '%SQL%' or v.key_skills ilike '%postgres%')
                     
'''

In [1385]:
# выводим результат
sql_postgr_vac_count = pd.read_sql_query(query_3_18, connection)
sql_postgr_vac_count

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/2869467579.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_postgr_vac_count = pd.read_sql_query(query_3_18, connection)


,vacancies_jun_count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [1386]:
# текст запроса
query_3_19 = f'''select
                     count(v.name) as vacancies_jun_count
                 from
                     VACANCIES as v
                 where
                     (
                     (name ilike '%data scientist%') or
                     (name ilike '%data science%') or
                     (name ilike '%исследователь данных%') or
                     ((name ilike '%ML%') and (name not ilike '%HTML%')) or
                     (name ilike '%machine learning%') or
                     (name ilike '%машинн%обучен%') and 
                     (key_skills is not null)
                     )
                     and (v.key_skills ilike '%python%')
                     
'''

In [1387]:
# выводим результат
python_vac_count = pd.read_sql_query(query_3_19, connection)
python_vac_count

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/1654453987.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  python_vac_count = pd.read_sql_query(query_3_19, connection)


,vacancies_jun_count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [1388]:
# текст запроса
query_3_20 = f'''select
                     avg(length(key_skills) - length(replace(key_skills, CHR(9), '')) + 1) as ds_key_skills_avg_count
                 from
                     VACANCIES as v
                 where
                     (
                     (name ilike '%data scientist%') or
                     (name ilike '%data science%') or
                     (name ilike '%исследователь данных%') or
                     ((name ilike '%ML%') and (name not ilike '%HTML%')) or
                     (name ilike '%machine learning%') or
                     (name ilike '%машинн%обучен%') and 
                     (key_skills is not null)
                     )                 
'''

In [1389]:
# выводим результат
key_skills_ds_count = pd.read_sql_query(query_3_20, connection)
key_skills_ds_count 

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/3033157507.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  key_skills_ds_count = pd.read_sql_query(query_3_20, connection)


,ds_key_skills_avg_count
0,6.546778


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [1390]:
# текст запроса
query_3_21 = f'''select
                     v.experience,
                     round(avg(coalesce((v.salary_from + v.salary_to)/2, v.salary_from, v.salary_to)),0)
                 from
                     VACANCIES as v
                 where
                     (
                     (name ilike '%data scientist%') or
                     (name ilike '%data science%') or
                     (name ilike '%исследователь данных%') or
                     ((name ilike '%ML%') and (name not ilike '%HTML%')) or
                     (name ilike '%machine learning%') or
                     (name ilike '%машинн%обучен%') and 
                     (key_skills is not null)
                     )
                 group by v.experience
                 order by 1                     
'''

In [1391]:
# текст запроса
key_skills_ds_count = pd.read_sql_query(query_3_21, connection)
key_skills_ds_count 

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/3800790036.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  key_skills_ds_count = pd.read_sql_query(query_3_21, connection)


,experience,round
0,Более 6 лет,157933.0
1,Нет опыта,74643.0
2,От 1 года до 3 лет,149499.0
3,От 3 до 6 лет,256454.0


Выводы по предметному анализу:
1. 1771 вакансия имеет отношение к данным.
2. Неплохое количество вакансий для начинающего дата-сайнтиста - 51.
3. Более востребован, все же, puthon, чем SQL и postgres. Под базы данных и СУБД 229 вакансий, под python - 357.
4. Для того, чтобы работать дата-сайнтистов необходимо, если до целого, в среднем 6 навыков.
5. В целом, для начинающего дата-сайнтиста по зарплатной вилке неплохой старт - более 70000 руб.

***

# Общий вывод по проекту

Общие выводы по проекту:
1. Вакансии для дата-сайнтистов представлены достаточно ярко и широко, что внушает определенный оптимизм для продолжения изучения специальности.
2. Достаточно широкая география IT вакансий, что позволяет привлекать не только менее требовательных по зарплате специалистов, но и, в случае желания переехать и отсутствия удаленной работы, самому найти себе подходящую (при этом не только в России).
3. Необходимость осваивать СУБД подтвердилась цифрами - 229 вакансий с необходимым ключевым навыком.
4. Представлено достаточно большое количество работодателей, которые предлагают вакансии дата-сайнтистам с неплохой зарплатной вилкой.
5. Дополнительные выводы приведены ниже.

Дополнительное исследование в отношении количества практических навыков, связанных с математикой

In [1392]:
# текст запроса
query_3_22 = f'''select
                     count(v.name) as vacancies_jun_count
                 from
                     VACANCIES as v
                 where
                     (
                     (name ilike '%data scientist%') or
                     (name ilike '%data science%') or
                     (name ilike '%исследователь данных%') or
                     ((name ilike '%ML%') and (name not ilike '%HTML%')) or
                     (name ilike '%machine learning%') or
                     (name ilike '%машинн%обучен%') and 
                     (key_skills is not null)
                     )
                     and (v.key_skills ilike '%математика%' or v.key_skills ilike '%math%')
                     
'''

In [1393]:
# выводим результат
math_skill_count = pd.read_sql_query(query_3_22, connection)
math_skill_count 

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/1666471020.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  math_skill_count = pd.read_sql_query(query_3_22, connection)


,vacancies_jun_count
0,10


Дополнительное исследование в отношении количества практических навык, связанных с soft skills

In [1394]:
# текст запроса
query_3_23 = f'''select
                     count(v.name) as vacancies_jun_count
                 from
                     VACANCIES as v
                 where
                     (
                     (name ilike '%data scientist%') or
                     (name ilike '%data science%') or
                     (name ilike '%исследователь данных%') or
                     ((name ilike '%ML%') and (name not ilike '%HTML%')) or
                     (name ilike '%machine learning%') or
                     (name ilike '%машинн%обучен%') and 
                     (key_skills is not null)
                     )
                     and (v.key_skills ilike '%soft skills%')
                     
'''

In [1395]:
# выводим результат
soft_skills_count = pd.read_sql_query(query_3_23, connection)
soft_skills_count

/var/folders/zy/nrgcv7_53yjdkhzk46rq81140000gn/T/ipykernel_34471/3931743984.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  soft_skills_count = pd.read_sql_query(query_3_23, connection)


,vacancies_jun_count
0,0


Выводы по дополнительным исследованиям:
1. Под ключевые навыки, связанные с математикой, выделено 10 вакансий.
2. В начале курса говорили, что для дата-сайнтиста важен навык soft skills, но вот, к сожалению, не отражен он никак в ключевых навыках) или же неверный запрос, или иной вариант написания.